# 10-4 ImageFolder (2)

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision
import torchvision.transforms as transforms # common image transformations.

In [13]:
# gpu 쓸 수 있으면 gpu 쓰고 안되면 cpu 사용
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

In [14]:
trans = transforms.Compose([
    transforms.ToTensor()
])
# torchvision.datasets.ImageFolder
# root/dog/xxx.png, root/dog/xxy.png...root/dog/[...]/xxz.png와 같이 정리되어있는 데이터셋 폴더를 불러온다
train_data = torchvision.datasets.ImageFolder(root='./custom_data/train_data', transform=trans) # 데이터셋을 오픈하며 trans 연산을 거침. 이경우 입력받은 데이터셋을 다 텐서로 만들어버림

In [15]:
data_loader = DataLoader(dataset = train_data, batch_size = 8, shuffle = True, num_workers=2) # DataLoader를 통해 학습에 사용할 수 있게 학습용 데이터셋을 생성

In [16]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29, 120),
            nn.ReLU(),
            nn.Linear(120,2)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.shape[0], -1) # Convolution 연산 후 Flatten 처리
        out = self.layer3(out)
        return out
        

In [17]:
#testing 
net = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = net(test_input)

In [18]:
# 최적화 방식, loss function 설정
optimizer = optim.Adam(net.parameters(), lr=0.00005)
loss_func = nn.CrossEntropyLoss().to(device)

In [19]:
total_batch = len(data_loader) # 총 배치 개수. 이번에는 69개다

epochs = 7
for epoch in range(epochs):
    avg_cost = 0.0
    for num, data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = net(imgs)
        loss = loss_func(out, labels)
        loss.backward()
        optimizer.step()
        
        avg_cost += loss / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')   

[Epoch:1] cost = 0.6551657915115356
[Epoch:2] cost = 0.4942275583744049
[Epoch:3] cost = 0.23501341044902802
[Epoch:4] cost = 0.06870721280574799
[Epoch:5] cost = 0.026442507281899452
[Epoch:6] cost = 0.013891587033867836
[Epoch:7] cost = 0.008641593158245087
Learning Finished!


In [11]:
torch.save(net.state_dict(), "./model.pth") # 학습한 모델을 저장

In [20]:
new_net = CNN().to(device)

In [21]:
new_net.load_state_dict(torch.load('./model.pth'))

<All keys matched successfully>

In [22]:
# 제대로 불러왔는지 확인
print(net.layer1[0])
print(new_net.layer1[0])

print(net.layer1[0].weight[0][0][0])
print(new_net.layer1[0].weight[0][0][0])

net.layer1[0].weight[0] == new_net.layer1[0].weight[0] 

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
tensor([-0.0915,  0.0031, -0.0173, -0.0214,  0.0929],
       grad_fn=<SelectBackward0>)
tensor([-0.0915,  0.0031, -0.0173, -0.0214,  0.0929],
       grad_fn=<SelectBackward0>)


tensor([[[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]],

        [[True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True],
         [True, True, True, True, True]]])

In [23]:
# 테스트용 데이터셋 가져오기
trans=torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='./custom_data/test_data', transform=trans) # 테스트 데이터를 불러오는데 이번에는 사이즈 조절 후 텐서로 만들어준다. 

In [24]:
# DataLoader로 만들어 테스트에 사용
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [25]:
# 테스트
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        
        correct_prediction = torch.argmax(prediction, 1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())

Accuracy: 1.0
